In [1]:
import os
import gc
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")
from time import sleep
import sistemas
import sei
import functions
from page import *

% reload_ext autoreload
% autoreload 2

In [2]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [3]:
sei_ = sei.login_sei(browser, usr, pwd)

In [4]:
sei_ = sei.Sei(sei_)

In [23]:
#scpx = webdriver.Ie()
scpx = sistemas.Scpx(scpx.driver)

In [21]:
cpf = '14804020870'
proc = r'53504.008508/2018-89'

In [7]:
sei_.itera_processos()

In [8]:
p = sei_.go_to_processo(proc)

In [24]:
scpx.consulta(cpf)

In [19]:
scpx.movimento_transferir(cpf, 'a', 'g', proc=proc)

In [25]:
scpx.movimento_cancelar(cpf)

In [21]:
p.is_open()

True

In [43]:
p.incluir_oficio('RC_OF_PPDESS')